## Анализ и визуализация данных на языке Python

### Агрегирование и групповые операции в pandas

1. Группировка данных
2. Агрегирование данных
3. Групповые операции над данными


In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

### Группировка данных

Под группировкой будем понимать разделение объекта pandas на части по одному или нескольким признакам. Такое объединение данных применяется для:
 - вычисления групповых статистик (минимум, максимум, сумма, и т.д.)
 - применения групповых преобразований
 - вычисления сводных таблиц и кросстабулирования

В pandas для группировки используется функция ```groupby()```. Эта функция сама по себе ничего не вычисляет, кроме промежуточных данных о ключах, по которым производится объединение. Функция возвращает объект, у которого есть методы для агрегатных вычислений.


In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.525737,0.252362
1,a,two,0.582630,-1.851616
2,b,one,0.902060,0.082388
3,b,two,1.634405,-0.207514
4,a,one,-0.763714,-1.449997


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
# С помощью атрибута groups можно посмотреть данные
grouped.groups, grouped.indices

({'a': [0, 1, 4], 'b': [2, 3]}, {'a': array([0, 1, 4]), 'b': array([2, 3])})

In [5]:
grouped.mean()

,data1
key1,
a,-0.235607
b,1.268233


Для группировки можно использовать несколько объектов Series:

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.644725
      two     0.582630
b     one     0.902060
      two     1.634405
Name: data1, dtype: float64

...или выполнять группировку по всему DataFrame'у, передав ключи в виде списка:

In [7]:
# это выражение должно вывести тот же результат, что и выражение выше
means = df.groupby(['key1', 'key2'])['data1'].mean()
means

key1  key2
a     one    -0.644725
      two     0.582630
b     one     0.902060
      two     1.634405
Name: data1, dtype: float64

...также можно указать один ключ. Обратите внимание: куда пропал ключ 'key2'?

In [8]:
means = df.groupby('key1').mean(numeric_only=True)
means

,data1,data2
key1,,
a,-0.235607,-1.016417
b,1.268233,-0.062563


Группировку можно проводить по любой оси. Например, сгруппируем данные по типу:

In [9]:
df.dtypes

,0
key1,object
key2,object
data1,float64
data2,float64


In [10]:
df.T

,0,1,2,3,4
key1,a,a,b,b,a
key2,one,two,one,two,one
data1,-0.525737,0.58263,0.90206,1.634405,-0.763714
data2,0.252362,-1.851616,0.082388,-0.207514,-1.449997


In [11]:
grouped = df.T.groupby(df.dtypes)
grouped.size()

,0
float64,2
object,2


Объект SeriesGroupBy поддерживает итерирование. Так, группы можно обходить в цикле ```for```. При этом итератор возвращает значение ключа и объект DataFrame группы:

In [12]:
for name, group in df.groupby('key1'):
    print(name)
    display(group)

a


,key1,key2,data1,data2
0,a,one,-0.525737,0.252362
1,a,two,0.582630,-1.851616
4,a,one,-0.763714,-1.449997


b


,key1,key2,data1,data2
2,b,one,0.902060,0.082388
3,b,two,1.634405,-0.207514


Сгруппируйте датафрейм df по признакам key1 и key2 и обратите внимание, что вернет первый параметр итератора.

In [13]:
for name, group in df.groupby(['key1', 'key2']):
    print("Multi index:", name)
    display(group)

Multi index: ('a', 'one')


,key1,key2,data1,data2
0,a,one,-0.525737,0.252362
4,a,one,-0.763714,-1.449997


Multi index: ('a', 'two')


,key1,key2,data1,data2
1,a,two,0.58263,-1.851616


Multi index: ('b', 'one')


,key1,key2,data1,data2
2,b,one,0.90206,0.082388


Multi index: ('b', 'two')


,key1,key2,data1,data2
3,b,two,1.634405,-0.207514


Также для группировки можно использовать словари и массивы. Рассмотрим пример:

In [14]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Alex', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.408233,-0.008602,1.704595,0.128611,0.712767
Steve,-0.412759,0.358938,1.305997,-0.713324,-0.629319
Alex,0.551627,0.130758,0.210241,1.441868,0.724188
Jim,-0.648606,0.253055,-1.076354,0.485608,1.835935
Travis,-0.266695,-1.839573,-0.512910,-0.432218,-3.035489


In [15]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [16]:
by_column = people.T.groupby(mapping)
by_column.sum().T

,blue,red
Joe,1.833206,1.112398
Steve,0.592673,-0.683141
Alex,1.652109,1.406574
Jim,-0.590747,1.440385
Travis,-0.945128,-5.141756


Также можно использовать для группировки функцию (например, ```len()```):

In [17]:
display(people)
people.groupby(len).sum()

,a,b,c,d,e
Joe,0.408233,-0.008602,1.704595,0.128611,0.712767
Steve,-0.412759,0.358938,1.305997,-0.713324,-0.629319
Alex,0.551627,0.130758,0.210241,1.441868,0.724188
Jim,-0.648606,0.253055,-1.076354,0.485608,1.835935
Travis,-0.266695,-1.839573,-0.512910,-0.432218,-3.035489


,a,b,c,d,e
3,-0.240373,0.244454,0.628241,0.614219,2.548702
4,0.551627,0.130758,0.210241,1.441868,0.724188
5,-0.412759,0.358938,1.305997,-0.713324,-0.629319
6,-0.266695,-1.839573,-0.512910,-0.432218,-3.035489


#### Множественный индекс

Вспомним группировку из предыдущего примера. Обратите внимание на индекс:

In [18]:
means = df.groupby(['key1', 'key2'])['data1'].mean()
means

key1  key2
a     one    -0.644725
      two     0.582630
b     one     0.902060
      two     1.634405
Name: data1, dtype: float64

DataFrame с множественным индексом можно задать явно:

In [19]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    ['specialized', 'santa cruz', 'gt', 'fuji', 'ninjago']],
                                    names=['cty', 'brand'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty            US                             JP          
brand specialized santa cruz        gt      fuji   ninjago
0        0.578456   1.801987  0.470967  0.276413 -0.195197
1       -0.108151   1.258612  1.554472  1.536901 -0.198797
2       -0.768429  -0.293295 -0.867246 -0.786312 -0.664519
3       -0.503254   0.369221  0.596092 -0.575425  0.341711

Для группировки указываем уровень в параметре ```level=```.

In [20]:
display(hier_df)
hier_df.T.groupby(level='cty').sum().T

cty            US                             JP          
brand specialized santa cruz        gt      fuji   ninjago
0        0.578456   1.801987  0.470967  0.276413 -0.195197
1       -0.108151   1.258612  1.554472  1.536901 -0.198797
2       -0.768429  -0.293295 -0.867246 -0.786312 -0.664519
3       -0.503254   0.369221  0.596092 -0.575425  0.341711

cty,JP,US
0,0.081215,2.851410
1,1.338105,2.704933
2,-1.450831,-1.928971
3,-0.233715,0.462058


### Агрегирование данных

Список оптимизированных агрегатных функций pandas:

- ```count()``` - количество отличных от NaN значений в группе
- ```sum()``` - сумма
- ```mean()``` - среднее (также по отличным от NaN значениям)
- ```median()``` - медиана (тоже по отличным от NaN)
- ```var()``` - дисперсия
- ```min()``` - минимум
- ```max()``` - максимум
- ```prod()``` - произведение
- ```first()``` - первый отличный от NaN в группе
- ```last()``` - последний отличный от NaN в группе

Также можно написать свою собственную функцию и передать ее в функцию ```agg()```. Она будет принимать на вход объект Series и возращать скалярное значение.


In [21]:
df

,key1,key2,data1,data2
0,a,one,-0.525737,0.252362
1,a,two,0.582630,-1.851616
2,b,one,0.902060,0.082388
3,b,two,1.634405,-0.207514
4,a,one,-0.763714,-1.449997


In [22]:
def mean_median(arr):
    return arr.mean() - arr.median()

df.drop('key2', axis=1).groupby('key1').agg(mean_median)

,data1,data2
key1,,
a,0.29013,0.43358
b,0.00000,0.00000


Также в ```agg()``` можно передавать список функций, стандартные функции передаются по именам:

In [23]:
df.drop('key2', axis=1).groupby('key1').agg(['mean', 'median', mean_median])

data1                           data2                      
          mean    median mean_median      mean    median mean_median
key1                                                                
a    -0.235607 -0.525737     0.29013 -1.016417 -1.449997     0.43358
b     1.268233  1.268233     0.00000 -0.062563 -0.062563     0.00000

In [24]:
# С помощью кортежа можно задать имена для столбцов
(
    df
     .drop('key2', axis=1)
     .groupby('key1')
     .agg([("Среднее", 'mean'),
           ("Медиана", 'median'),
           ("Среднее-медиана", mean_median)
          ])
)

data1                               data2                          
       Среднее   Медиана Среднее-медиана   Среднее   Медиана Среднее-медиана
key1                                                                        
a    -0.235607 -0.525737         0.29013 -1.016417 -1.449997         0.43358
b     1.268233  1.268233         0.00000 -0.062563 -0.062563         0.00000

## Набор данных `tips.csv(чаевыe)`

In [25]:
tips = pd.read_csv('data/tips.csv')
# Добавим процент чаевых от общей суммы счета
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


### Групповые операции над данными

Функция apply позволяет применить заданную функцию к каждой группе. И более того, она может вернуть векторное значение.

In [26]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column, ascending=False)[:n]

top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525


Так мы можем вывести самых щедрых на чаевые из групп "курящие/некурящие":

In [27]:
tips.groupby('smoker').apply(top, include_groups=False)

total_bill   tip     sex   day    time  size   tip_pct
smoker                                                            
No     232       11.61  3.39    Male   Sat  Dinner     2  0.291990
       149        7.51  2.00    Male  Thur   Lunch     2  0.266312
       51        10.29  2.60  Female   Sun  Dinner     2  0.252672
       185       20.69  5.00    Male   Sun  Dinner     5  0.241663
       88        24.71  5.85    Male  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Male   Sun  Dinner     2  0.710345
       178        9.60  4.00  Female   Sun  Dinner     2  0.416667
       67         3.07  1.00  Female   Sat  Dinner     1  0.325733
       183       23.17  6.50    Male   Sun  Dinner     4  0.280535
       109       14.31  4.00  Female   Sat  Dinner     2  0.279525

Параметры к нашей функции можно передать через ту же функцию ```apply()```

In [28]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill', include_groups=False)

total_bill    tip     sex    time  size   tip_pct
smoker day                                                        
No     Fri  94        22.75   3.25  Female  Dinner     2  0.142857
       Sat  212       48.33   9.00    Male  Dinner     4  0.186220
       Sun  156       48.17   5.00    Male  Dinner     6  0.103799
       Thur 142       41.19   5.00    Male   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Male  Dinner     4  0.117750
       Sat  170       50.81  10.00    Male  Dinner     3  0.196812
       Sun  182       45.35   3.50    Male  Dinner     3  0.077178
       Thur 197       43.11   5.00  Female   Lunch     4  0.115982

Избавиться от индекса в выводе данных можно с помощью параметра ```group_keys=False```

In [29]:
(
    tips
     .groupby(['smoker', 'day'], group_keys=False)
     .apply(top, n=1, column='total_bill', include_groups=False)
)

,total_bill,tip,sex,time,size,tip_pct
94,22.75,3.25,Female,Dinner,2,0.142857
212,48.33,9.00,Male,Dinner,4,0.186220
156,48.17,5.00,Male,Dinner,6,0.103799
142,41.19,5.00,Male,Lunch,5,0.121389
95,40.17,4.73,Male,Dinner,4,0.117750
170,50.81,10.00,Male,Dinner,3,0.196812
182,45.35,3.50,Male,Dinner,3,0.077178
197,43.11,5.00,Female,Lunch,4,0.115982
